# HydroSOS - Persistence and Hydrological Analogues method

In this notebook, we explore the Persistance and Hydrological Analogue method

## Import the python libraries

In [1]:
%reset -f

In [2]:
# Importing the libraries
import pandas as pd
import seaborn as sns
import numpy as np
import calendar
import datetime
import matplotlib.pyplot as plt
import math
from scipy import stats
plt.style.use('classic')
%matplotlib inline

from IPython.display import HTML

sns.set()

In [3]:
# install a conda package in the current Jupyter kernel
#import sys
#!conda install --yes --prefix {sys.prefix} missingno

In [4]:
# import missingno

### We create function for percentile definition

In [5]:
# Quantile Function Definitions
def q1(x):
    return x.quantile(0.28)

def q2(x):
    return x.median()

def q3(x):
    return x.quantile(0.72)

def q5(x):
    return x.quantile(0.05)

def q95(x):
    return x.quantile(0.95)

def q87(x):
    return x.quantile(0.87)

def q13(x):
    return x.quantile(0.13)


We create a function called add_month that add the number of months based on an input date. This will be used in the Jack-Knife Validation

In [6]:
def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year,month)[1])
    return datetime.date(year, month, day)

the monthly anomalies of the most recently past months are compared with all possible historical sequences of anomalies covering the same months of the year. That is, if the recent past covers, say, the months February to July, then potential analogues are sought only in the February to July sequences of the historical record

Define a Root Mean Square Error Function. This function will be used to select the analogues years

In [7]:
def rmse(predictions,targets):
    return np.around(((predictions - targets) ** 2).mean() ** 0.5,4)

## Import Data 

In this example we import the daily discharge data from Fray Marcos station, from 01 Jan 1980 to 31 Jan 2023. We import and then visualize head and tail of dataframe, plot timeseries and also make boxplot for daily discharge grouped by month.

In [8]:
MONTHLY_ANOMALY = pd.read_csv('../data/santalucia_caudales_abr2023_MA.csv',parse_dates=['Month'],index_col="Month",na_values="")

In [9]:
MONTHLY_ANOMALY

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Month,,,,,,,,,,,,,,,,,,,,,
1,-2.123750,-0.308179,-1.551073,-0.796712,0.727208,-1.244110,-0.670261,-0.504405,0.818331,-1.280020,...,NaN,0.204800,0.814189,0.200969,0.324539,NaN,-0.385739,-0.230398,1.141266,-0.810341
2,0.316909,0.511895,0.235052,0.100800,2.454052,-1.259258,-0.370705,-0.773939,0.187623,-1.368724,...,NaN,-0.394508,-0.887051,0.577828,-0.659732,NaN,-0.742382,0.860497,-0.252152,-0.951652
3,1.120851,-0.362955,-0.375811,-0.951718,1.193772,0.744396,-0.898463,0.402005,1.742555,-0.994765,...,NaN,-0.677517,-1.096631,-0.545900,0.112274,-0.449993,-1.059530,0.710564,-0.389803,-1.550890
4,1.200881,-0.162872,-0.336847,-0.364767,-0.160803,0.583685,-0.569182,-0.573368,0.695763,-0.983002,...,NaN,NaN,2.118149,-0.969378,-0.979009,-1.319303,-1.188353,0.670950,-0.613546,-1.664257
5,1.373637,1.804841,0.930711,-0.734193,0.722645,0.782769,0.315669,-0.976373,-0.788636,-1.846345,...,NaN,-1.283716,0.049850,0.107734,-0.537679,0.053286,-1.607545,-0.087954,-0.178636,NaN
6,0.796236,-0.163576,1.175735,0.284660,1.159690,0.689027,0.508366,-1.715410,-1.236878,-2.084692,...,NaN,-1.814261,-0.301783,-0.850826,-0.520173,1.715769,0.585062,1.115231,-1.331390,NaN
7,0.669569,0.327620,0.605223,-0.332697,1.575762,0.619012,0.095685,-0.033314,-0.667932,-1.610597,...,NaN,-2.458694,1.464433,0.166312,0.883619,-0.018995,-0.531656,-0.129605,0.197415,NaN
8,0.585540,0.927440,0.726022,1.307074,-0.886873,0.710619,2.020854,-0.018238,-0.082788,-0.035092,...,NaN,0.983076,0.589616,1.700595,NaN,0.098847,-0.737193,-0.005090,-1.282212,NaN
9,-0.481964,0.523972,0.384146,1.468819,-0.124697,0.380196,0.927865,-0.596046,-1.185904,-1.516816,...,NaN,-1.124536,0.693663,1.629718,NaN,0.482405,0.104265,0.059747,-1.975643,NaN


In [12]:
DISCHARGE_MONTHLY = pd.read_csv('../data/santalucia_caudales_abr2023_DA.csv',parse_dates=['Fecha'],index_col="Fecha",na_values="")

In [13]:
DISCHARGE_MONTHLY

,Discharge,Year,Month,water_year,Q_to_log,Anomaly_Qlog
Fecha,,,,,,
1980-01-31,1.198613,1980,1,1980,0.181165,-2.123750
1980-02-29,28.386414,1980,2,1980,3.345911,0.316909
1980-03-31,86.782323,1980,3,1980,4.463403,1.120851
1980-04-30,172.187367,1980,4,1981,5.148583,1.200881
1980-05-31,275.560258,1980,5,1981,5.618806,1.373637
...,...,...,...,...,...,...
2022-12-31,5.969037,2022,12,2023,1.786586,-1.063870
2023-01-31,5.419419,2023,1,2023,1.689989,-0.810341
2023-02-28,5.557464,2023,2,2023,1.715142,-0.951652


In [14]:
YEAR_HINDCAST_SELECTED = np.unique(DISCHARGE_MONTHLY['Year'])
# rename columns change str for int
MONTHLY_ANOMALY.columns = YEAR_HINDCAST_SELECTED

In [26]:
YEAR_HINDCAST_SELECTED

array([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
      dtype=int64)

In [31]:
range_analysis = range(YEAR_HINDCAST_SELECTED[0],YEAR_HINDCAST_SELECTED[-1])

In [32]:
LOG_DISCHARGE_MONTHLY_PIVOT = pd.pivot_table(DISCHARGE_MONTHLY, index=['Month'],columns=['Year'], values=['Q_to_log'],dropna=False)
LOG_DISCHARGE_MONTHLY_PIVOT.columns = YEAR_HINDCAST_SELECTED

In [33]:
LOG_DISCHARGE_MONTHLY_PIVOT

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Month,,,,,,,,,,,,,,,,,,,,,
1,0.181165,2.266864,0.839047,1.705645,3.456299,1.191681,1.850911,2.041442,3.560980,1.150429,...,NaN,2.856166,3.556222,2.851765,2.993721,NaN,2.177764,2.356218,3.931962,1.689989
2,3.345911,3.596570,3.240681,3.068096,6.093265,1.319706,2.461964,1.943596,3.179709,1.178985,...,NaN,2.431365,1.798187,3.681329,2.090412,NaN,1.984164,4.044708,2.614367,1.715142
3,4.463403,2.653676,2.637997,1.935590,4.552342,4.004259,2.000542,3.586661,5.221666,1.883088,...,NaN,2.270020,1.758847,2.430547,3.233290,2.547520,1.804097,3.962996,2.620931,1.204808
4,5.148583,3.169494,2.917019,2.876502,3.172495,4.252904,2.579853,2.573778,4.415552,1.979313,...,NaN,NaN,6.479732,1.999085,1.985108,1.491270,1.681306,4.379543,2.515471,0.990670
5,5.618806,6.181355,5.040966,2.868932,4.769523,4.847961,4.238582,2.552985,2.797907,1.418020,...,NaN,2.152025,3.891795,3.967311,3.125305,3.896278,1.729558,3.712016,3.593711,NaN
6,5.431652,4.226532,5.908143,4.789328,5.887999,5.297043,5.070209,2.278081,2.878916,1.814417,...,NaN,2.153965,4.053002,3.363635,3.778796,6.586199,5.166506,5.832176,2.760248,NaN
7,5.315642,4.974052,5.251364,4.314426,6.220885,5.265138,4.742359,4.613495,3.979544,3.037866,...,NaN,2.190658,6.109673,4.812913,5.529467,4.627800,4.115676,4.517306,4.843983,NaN
8,5.119868,5.464556,5.261496,5.847285,3.635448,5.245967,6.566886,4.511166,4.446090,4.494176,...,NaN,5.520645,5.123977,6.244016,NaN,4.629206,3.786349,4.524422,3.236886,NaN
9,4.117590,5.092622,4.957092,6.008443,4.463882,4.953263,5.484107,4.007013,3.435276,3.114530,...,NaN,3.494759,5.257100,6.164399,NaN,5.052332,4.685809,4.642659,2.669799,NaN


In [34]:
ANOMALY_ANALOGUES = MONTHLY_ANOMALY[range_analysis]
QLOG_ANALOGUES = LOG_DISCHARGE_MONTHLY_PIVOT[range_analysis]
HTML(ANOMALY_ANALOGUES.to_html())

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-2.123750,-0.308179,-1.551073,-0.796712,0.727208,-1.244110,-0.670261,-0.504405,0.818331,-1.280020,-1.217702,-0.332189,-0.151861,-0.391850,1.297849,-0.771544,-0.773628,-0.609809,1.599805,2.014703,-0.752051,NaN,0.168765,1.662446,0.220363,0.214728,2.427332,-0.405921,-0.069919,-0.275124,1.082403,-0.171486,NaN,0.908379,NaN,0.204800,0.814189,0.200969,0.324539,NaN,-0.385739,-0.230398,1.141266
2,0.316909,0.511895,0.235052,0.100800,2.454052,-1.259258,-0.370705,-0.773939,0.187623,-1.368724,0.695973,-0.834223,-0.451418,1.702978,NaN,0.080346,-1.114598,-0.735161,-0.535340,2.378751,-1.082684,NaN,-0.004572,0.958376,-0.386063,0.492906,-0.137558,-0.672549,-0.303736,0.213084,2.828374,-0.399055,NaN,-0.278387,NaN,-0.394508,-0.887051,0.577828,-0.659732,NaN,-0.742382,0.860497,-0.252152
3,1.120851,-0.362955,-0.375811,-0.951718,1.193772,0.744396,-0.898463,0.402005,1.742555,-0.994765,0.368636,-0.618578,-0.621814,-0.434609,0.184724,-0.028342,-1.316506,-0.671217,-0.821700,1.549766,-1.341889,1.498400,2.527140,0.214817,-0.607739,-0.278698,0.094414,2.332151,0.803340,0.482773,0.549333,-0.536844,NaN,NaN,NaN,-0.677517,-1.096631,-0.545900,0.112274,-0.449993,-1.059530,0.710564,-0.389803
4,1.200881,-0.162872,-0.336847,-0.364767,-0.160803,0.583685,-0.569182,-0.573368,0.695763,-0.983002,1.740413,-0.820711,-0.109710,0.941890,-0.084077,1.142900,0.569727,-1.152541,-0.223646,0.620392,NaN,-0.303012,2.078428,-0.689754,-0.127231,1.688282,-0.487829,1.956936,-0.653459,-0.651857,0.446257,-0.679586,-0.586552,NaN,NaN,NaN,2.118149,-0.969378,-0.979009,-1.319303,-1.188353,0.670950,-0.613546
5,1.373637,1.804841,0.930711,-0.734193,0.722645,0.782769,0.315669,-0.976373,-0.788636,-1.846345,0.055118,-0.026225,0.897623,0.929453,1.142730,0.193611,-0.909332,-1.229191,-0.437847,-0.628618,2.064755,-0.136805,1.454932,0.670987,0.324343,1.216989,-0.986345,1.713232,-0.864665,-1.028210,-0.305526,-0.990349,-1.220725,NaN,NaN,-1.283716,0.049850,0.107734,-0.537679,0.053286,-1.607545,-0.087954,-0.178636
6,0.796236,-0.163576,1.175735,0.284660,1.159690,0.689027,0.508366,-1.715410,-1.236878,-2.084692,0.182126,0.362392,0.517625,0.271876,-0.628118,0.309844,-1.732159,-0.216433,-0.312289,0.386102,0.505539,1.243150,0.734981,1.084724,-0.109196,1.533202,1.317109,-0.217848,-0.893280,-0.582289,0.612672,NaN,-1.206965,-1.173551,NaN,-1.814261,-0.301783,-0.850826,-0.520173,1.715769,0.585062,1.115231,-1.331390
7,0.669569,0.327620,0.605223,-0.332697,1.575762,0.619012,0.095685,-0.033314,-0.667932,-1.610597,-1.619038,0.623227,0.285137,-0.455937,-0.508600,0.499323,-2.199121,-1.788752,0.929367,1.672127,1.241263,-0.321207,0.434812,0.506555,-0.236939,0.311917,0.214526,-1.384830,-0.367381,1.067286,1.057527,NaN,-0.782419,NaN,NaN,-2.458694,1.464433,0.166312,0.883619,-0.018995,-0.531656,-0.129605,0.197415
8,0.585540,0.927440,0.726022,1.307074,-0.886873,0.710619,2.020854,-0.018238,-0.082788,-0.035092,-1.997518,0.865586,-0.708144,0.028734,0.605638,-1.407229,-2.770442,-0.163174,-0.543317,-0.011291,NaN,0.712015,0.528228,0.476646,-0.759065,-1.105045,0.331471,-0.278306,-1.034313,-0.365984,0.993314,NaN,NaN,NaN,NaN,0.983076,0.589616,1.700595,NaN,0.098847,-0.737193,-0.005090,-1.282212
9,-0.481964,0.523972,0.384146,1.468819,-0.124697,0.380196,0.927865,-0.596046,-1.185904,-1.516816,-1.517861,0.693688,0.532099,0.656601,0.844961,-1.777712,0.490324,-1.238533,-0.830146,0.421248,NaN,0.039467,0.323237,1.584296,0.270234,-0.408260,-1.201608,1.047919,-0.988840,-0.336547,1.746241,NaN,NaN,NaN,NaN,-1.124536,0.693663,1.629718,NaN,0.482405,0.104265,0.059747,-1.975643


In [35]:
ANOMALY_ANALOGUE_TS = pd.melt(ANOMALY_ANALOGUES.reset_index(), id_vars='Month',var_name=['Year'],ignore_index = True)
ANOMALY_ANALOGUE_TS.columns = ['Month','Year','Anomaly_Qlog']
HTML(ANOMALY_ANALOGUE_TS.head(6).to_html())

,Month,Year,Anomaly_Qlog
0,1,1980,-2.123750
1,2,1980,0.316909
2,3,1980,1.120851
3,4,1980,1.200881
4,5,1980,1.373637
5,6,1980,0.796236


In [36]:
QLOG_ANALOGUE_TS = pd.melt(QLOG_ANALOGUES.reset_index(), id_vars='Month',var_name=['Year'],ignore_index = True)
QLOG_ANALOGUE_TS.columns = ['Month','Year','Qlog']
HTML(QLOG_ANALOGUE_TS.head(6).to_html())

,Month,Year,Qlog
0,1,1980,0.181165
1,2,1980,3.345911
2,3,1980,4.463403
3,4,1980,5.148583
4,5,1980,5.618806
5,6,1980,5.431652


In [37]:
ANOMALY_ANALOGUE_TS['DATE'] = pd.to_datetime(ANOMALY_ANALOGUE_TS[['Year', 'Month']].assign(DAY=1))
HTML(ANOMALY_ANALOGUE_TS.head(6).to_html(index=False))

Month,Year,Anomaly_Qlog,DATE
1,1980,-2.123750,1980-01-01
2,1980,0.316909,1980-02-01
3,1980,1.120851,1980-03-01
4,1980,1.200881,1980-04-01
5,1980,1.373637,1980-05-01
6,1980,0.796236,1980-06-01


In [38]:
######## PARTE CENTRAL

## HINDCAST PARA FORECAST_LENGHT EN MESES

In [39]:
## define Dana last months of observation
DANA = 9
## define forecast lenght in months
FORECAST_LENGTH = 3
##

In [40]:
##
# Toma como argumentos
# range_analysis - > YEAR_ANALYSIS
# QLOG_ANALOGUES
# QLOG_ANALOGUE_TS
# ANOMALY_ANALOGUES
# ANOMALY_ANALOGUE_TS
# MONTH_ANALYSIS
# DANA
# FORECAST_LENGTH
##


YEAR_ANALYSIS = range_analysis # definir al menos para tener DANA meses para atras con datos..

MONTH_ANALYSIS = range(1,13,1)
df = pd.DataFrame()
ANOMALY_HISTORICAL = pd.DataFrame()
RMSE = []
ANO_TARG = []
ANO_PRED = []
MON_TARG = []
MON_PRED = []
TARG = []
PRED = []
OBS_ANOM = []

MEAN_ANOM_HIND = []
STD_ANOM_HIND = []
Q1_ANOM_HIND = []
Q3_ANOM_HIND = []

MEAN_QLOG_HIND = []
STD_QLOG_HIND = []

##
print('Dana selected: ',DANA, 'months')
print('Forecast length selected: ', FORECAST_LENGTH , 'months')
##


for y in YEAR_ANALYSIS:    
            
    
    for m in MONTH_ANALYSIS:
        
        print('Year:',y,"-","End-Month of observation:",m)
        # define end month and year based on the for-loop
        END_DATE_HIST = datetime.date.today().replace(year=y, month=m, day=1)

        # define initial date according to Dana past months of observation
        FIRST_DATE_HIST = add_months(END_DATE_HIST,-DANA)
        
        # Define forecast date
        FORECAST_DATE = add_months(END_DATE_HIST,FORECAST_LENGTH)       
        print('Forecast Year:',FORECAST_DATE.year,"-"," Forecast Month:",FORECAST_DATE.month)

        
        if FORECAST_DATE.year <= YEAR_ANALYSIS[-1]:
            # From the Qlog values, drop the monthly values for the year of analysis and mantain the remaining years. 
            # This is the Qlog hindcast series
            # print('Dropping year:',y,"-","End-Month:",m)
            print('Dropping year:',y," - get series for analysis.")
            QLOG_HINDCAST_TARG_ANALOGUE = QLOG_ANALOGUES.drop([FORECAST_DATE.year],axis=1)
            QLOG_HINDCAST_TARG_ANALOGUE_TS = QLOG_ANALOGUE_TS[QLOG_ANALOGUE_TS['Year'] != FORECAST_DATE.year]

            # From the monthly anomaly values, drop the monthly values for the year of analysis and mantain the remaining years. 
            # This is the anomaly hindcast series
            ANOMALY_HINDCAST_TARG_ANALOGUE = ANOMALY_ANALOGUES.drop([FORECAST_DATE.year],axis=1)
            ANOMALY_HINDCAST_TARG_ANALOGUE_TS = ANOMALY_ANALOGUE_TS[ANOMALY_ANALOGUE_TS['Year'] != FORECAST_DATE.year]
        
        else:
            print('Forecast Year out of range')
            break

            
        
        # query in the original anomaly time serie the first date and end_date
        ANOMALY_TARGET = ANOMALY_ANALOGUE_TS.query('DATE > @FIRST_DATE_HIST & DATE <= @END_DATE_HIST')
        print(ANOMALY_TARGET)
        
        # Check if there are enough months in the past
        if ANOMALY_TARGET.index.size < DANA:
            print('There are not enough months in the past')
            print('')
        
        else:
            print('There are enough months in the past')
            print(f"Target period: {add_months(FIRST_DATE_HIST,1).strftime('%m-%Y')} to {(END_DATE_HIST.strftime('%m-%Y'))}")
            
            # Define a list with the targets months
            MONTHS_TARG = pd.date_range(add_months(FIRST_DATE_HIST,1),add_months(END_DATE_HIST,1),freq='M').month.to_list()
            
            # Get the anomaly hindcast target series
            ANOMALY_HINDCAST_TARG = ANOMALY_ANALOGUE_TS.drop(ANOMALY_TARGET.index)
            ANOMALY_HINDCAST_TARG = ANOMALY_HINDCAST_TARG.query('Month == @MONTHS_TARG').reset_index()
            
            # Get the anomaly target series
            df['target'] = ANOMALY_TARGET["Anomaly_Qlog"].values.round(5)

            # Adjust anomaly hindcast target series index
            if MONTHS_TARG[0]>=MONTHS_TARG[-1]:
                print('Target period includes two years')
                print('')
                if ANOMALY_HINDCAST_TARG['Month'][0] != MONTHS_TARG[0]:
                    ANOMALY_HINDCAST_TARG.drop([*range(0,ANOMALY_HINDCAST_TARG[ANOMALY_HINDCAST_TARG['Month']==MONTHS_TARG[0]].index.min())],inplace=True)
                    ANOMALY_HINDCAST_TARG.reset_index(drop=True)
                    # print(len(ANOMALY_HINDCAST_TARG.index)/DANA)

            else:
                print('Target period includes only one year')
                print('')
                pass

            # Iterate over de anomaly hindcast target where are the potential analogues
            for i in range(0, int(len(ANOMALY_HINDCAST_TARG.index)/DANA)):

                
                # create a dataframe (df) for the "preditec" (analogue) and "target" (current year)
                df['predicted'] = ANOMALY_HINDCAST_TARG.iloc[i*DANA:(i+1)*DANA,1:-1]["Anomaly_Qlog"].values.round(5)
                
                # save month and year value of target series
                ANO_TARG.append(f"{str(ANOMALY_HINDCAST_TARG.iloc[i*DANA:(i+1)*DANA,1:-1]['Year'].tolist()[0])}-{str(ANOMALY_HINDCAST_TARG.iloc[i*DANA:(i+1)*DANA,1:-1]['Year'].tolist()[-1])}")
                MON_TARG.append(MONTHS_TARG)
                
                # save month and year value of predicted series
                ANO_PRED.append(add_months(END_DATE_HIST, FORECAST_LENGTH).year)
                MON_PRED.append(add_months(END_DATE_HIST, FORECAST_LENGTH).month)            

                # define the date to get the anomaly value of the actual month of the potential analogue
                DATE_DANA=datetime.date.today().replace(year=ANOMALY_HINDCAST_TARG.iloc[i*DANA:(i+1)*DANA]['Year'].values[-1], month=ANOMALY_HINDCAST_TARG.iloc[i*DANA:(i+1)*DANA]['Month'].values[-1], day=1)
                
                # get the anomaly value of the following month of the potential analogue
                ANOMALY_OBS = ANOMALY_ANALOGUE_TS.query('DATE == @add_months(@DATE_DANA,@FORECAST_LENGTH)')['Anomaly_Qlog'].values
                # print(ANOMALY_OBS)
                
                # get the mean and standard deviation of monthly anomaly series use for hindcast necessary for rescale de forecast
                MEAN_ANOM_HIND.append(ANOMALY_HINDCAST_TARG_ANALOGUE.loc[add_months(DATE_DANA,FORECAST_LENGTH).month].mean())
                STD_ANOM_HIND.append(ANOMALY_HINDCAST_TARG_ANALOGUE.loc[add_months(DATE_DANA,FORECAST_LENGTH).month].std())           
                
                # get statistic of monthly anomaly serie use for hindcast necesary contingency tables
                Q1_ANOM_HIND.append(q1(ANOMALY_HINDCAST_TARG_ANALOGUE.loc[add_months(DATE_DANA,FORECAST_LENGTH).month]))
                Q3_ANOM_HIND.append(q3(ANOMALY_HINDCAST_TARG_ANALOGUE.loc[add_months(DATE_DANA,FORECAST_LENGTH).month]))  
                
                # get the mean and standard deviation of monthly qlog series use for hindcast 
                MEAN_QLOG_HIND.append(QLOG_HINDCAST_TARG_ANALOGUE.loc[add_months(DATE_DANA,FORECAST_LENGTH).month].mean())
                STD_QLOG_HIND.append(QLOG_HINDCAST_TARG_ANALOGUE.loc[add_months(DATE_DANA,FORECAST_LENGTH).month].std())

                
                try:
                    if math.isnan(ANOMALY_OBS):
                        #print('Next month to end-month is nan')
                        RMSE_val = np.nan
                        RMSE.append(RMSE_val)
                        TARG.append(df['target'].values.tolist())
                        PRED.append(df['predicted'].values.tolist())
                        OBS_ANOM.append(np.nan)
                    else:
                        #print('Next month to end-month is not empty')
                        if df['predicted'].isnull().values.any() == False:
                            # calculate the RMSE
                            RMSE_val = rmse(df['predicted'],df['target'])
                            # append in the empty lists
                            RMSE.append(RMSE_val)
                            TARG.append(df['target'].values.tolist())
                            PRED.append(df['predicted'].values.tolist())
                            OBS_ANOM.append(ANOMALY_OBS.round(5)[0])
                        else:
                            RMSE_val = np.nan
                            RMSE.append(RMSE_val)
                            TARG.append(df['target'].values.tolist())
                            PRED.append(df['predicted'].values.tolist())
                            OBS_ANOM.append(np.nan)
                except:
                    if ANOMALY_OBS.size == 0:
                        #print('Next month to end-month is empty')
                        RMSE_val = np.nan
                        RMSE.append(RMSE_val)
                        TARG.append(df['target'].values.tolist())
                        PRED.append(df['predicted'].values.tolist())
                        OBS_ANOM.append(np.nan)      


Dana selected:  9 months
Forecast length selected:  3 months
Year: 1980 - End-Month of observation: 1
Forecast Year: 1980 -  Forecast Month: 4
Dropping year: 1980  - get series for analysis.
   Month  Year  Anomaly_Qlog       DATE
0      1  1980      -2.12375 1980-01-01
There are not enough months in the past

Year: 1980 - End-Month of observation: 2
Forecast Year: 1980 -  Forecast Month: 5
Dropping year: 1980  - get series for analysis.
   Month  Year  Anomaly_Qlog       DATE
0      1  1980     -2.123750 1980-01-01
1      2  1980      0.316909 1980-02-01
There are not enough months in the past

Year: 1980 - End-Month of observation: 3
Forecast Year: 1980 -  Forecast Month: 6
Dropping year: 1980  - get series for analysis.
   Month  Year  Anomaly_Qlog       DATE
0      1  1980     -2.123750 1980-01-01
1      2  1980      0.316909 1980-02-01
2      3  1980      1.120851 1980-03-01
There are not enough months in the past

Year: 1980 - End-Month of observation: 4
Forecast Year: 1980 -  Fo

Year: 1981 - End-Month of observation: 5
Forecast Year: 1981 -  Forecast Month: 8
Dropping year: 1981  - get series for analysis.
    Month  Year  Anomaly_Qlog       DATE
8       9  1980     -0.481964 1980-09-01
9      10  1980      0.640823 1980-10-01
10     11  1980     -0.107321 1980-11-01
11     12  1980      0.958023 1980-12-01
12      1  1981     -0.308179 1981-01-01
13      2  1981      0.511895 1981-02-01
14      3  1981     -0.362955 1981-03-01
15      4  1981     -0.162872 1981-04-01
16      5  1981      1.804841 1981-05-01
There are enough months in the past
Target period: 09-1980 to 05-1981
Target period includes two years

Year: 1981 - End-Month of observation: 6
Forecast Year: 1981 -  Forecast Month: 9
Dropping year: 1981  - get series for analysis.
    Month  Year  Anomaly_Qlog       DATE
9      10  1980      0.640823 1980-10-01
10     11  1980     -0.107321 1980-11-01
11     12  1980      0.958023 1980-12-01
12      1  1981     -0.308179 1981-01-01
13      2  1981      

Year: 1982 - End-Month of observation: 6
Forecast Year: 1982 -  Forecast Month: 9
Dropping year: 1982  - get series for analysis.
    Month  Year  Anomaly_Qlog       DATE
21     10  1981     -0.748537 1981-10-01
22     11  1981     -1.188138 1981-11-01
23     12  1981      0.398420 1981-12-01
24      1  1982     -1.551073 1982-01-01
25      2  1982      0.235052 1982-02-01
26      3  1982     -0.375811 1982-03-01
27      4  1982     -0.336847 1982-04-01
28      5  1982      0.930711 1982-05-01
29      6  1982      1.175735 1982-06-01
There are enough months in the past
Target period: 10-1981 to 06-1982
Target period includes two years

Year: 1982 - End-Month of observation: 7
Forecast Year: 1982 -  Forecast Month: 10
Dropping year: 1982  - get series for analysis.
    Month  Year  Anomaly_Qlog       DATE
22     11  1981     -1.188138 1981-11-01
23     12  1981      0.398420 1981-12-01
24      1  1982     -1.551073 1982-01-01
25      2  1982      0.235052 1982-02-01
26      3  1982     

Year: 1983 - End-Month of observation: 7
Forecast Year: 1983 -  Forecast Month: 10
Dropping year: 1983  - get series for analysis.
    Month  Year  Anomaly_Qlog       DATE
34     11  1982           NaN 1982-11-01
35     12  1982           NaN 1982-12-01
36      1  1983     -0.796712 1983-01-01
37      2  1983      0.100800 1983-02-01
38      3  1983     -0.951718 1983-03-01
39      4  1983     -0.364767 1983-04-01
40      5  1983     -0.734193 1983-05-01
41      6  1983      0.284660 1983-06-01
42      7  1983     -0.332697 1983-07-01
There are enough months in the past
Target period: 11-1982 to 07-1983
Target period includes two years

Year: 1983 - End-Month of observation: 8
Forecast Year: 1983 -  Forecast Month: 11
Dropping year: 1983  - get series for analysis.
    Month  Year  Anomaly_Qlog       DATE
35     12  1982           NaN 1982-12-01
36      1  1983     -0.796712 1983-01-01
37      2  1983      0.100800 1983-02-01
38      3  1983     -0.951718 1983-03-01
39      4  1983    

Year: 1984 - End-Month of observation: 8
Forecast Year: 1984 -  Forecast Month: 11
Dropping year: 1984  - get series for analysis.
    Month  Year  Anomaly_Qlog       DATE
47     12  1983     -0.256680 1983-12-01
48      1  1984      0.727208 1984-01-01
49      2  1984      2.454052 1984-02-01
50      3  1984      1.193772 1984-03-01
51      4  1984     -0.160803 1984-04-01
52      5  1984      0.722645 1984-05-01
53      6  1984      1.159690 1984-06-01
54      7  1984      1.575762 1984-07-01
55      8  1984     -0.886873 1984-08-01
There are enough months in the past
Target period: 12-1983 to 08-1984
Target period includes two years

Year: 1984 - End-Month of observation: 9
Forecast Year: 1984 -  Forecast Month: 12
Dropping year: 1984  - get series for analysis.
    Month  Year  Anomaly_Qlog       DATE
48      1  1984      0.727208 1984-01-01
49      2  1984      2.454052 1984-02-01
50      3  1984      1.193772 1984-03-01
51      4  1984     -0.160803 1984-04-01
52      5  1984    

...

Year: 2018 - End-Month of observation: 3
Forecast Year: 2018 -  Forecast Month: 6
Dropping year: 2018  - get series for analysis.
     Month  Year  Anomaly_Qlog       DATE
450      7  2017      0.166312 2017-07-01
451      8  2017      1.700595 2017-08-01
452      9  2017      1.629718 2017-09-01
453     10  2017     -0.108481 2017-10-01
454     11  2017     -0.337047 2017-11-01
455     12  2017     -0.803140 2017-12-01
456      1  2018      0.324539 2018-01-01
457      2  2018     -0.659732 2018-02-01
458      3  2018      0.112274 2018-03-01
There are enough months in the past
Target period: 07-2017 to 03-2018
Target period includes two years

Year: 2018 - End-Month of observation: 4
Forecast Year: 2018 -  Forecast Month: 7
Dropping year: 2018  - get series for analysis.
     Month  Year  Anomaly_Qlog       DATE
451      8  2017      1.700595 2017-08-01
452      9  2017      1.629718 2017-09-01
453     10  2017     -0.108481 2017-10-01
454     11  2017     -0.337047 2017-11-01
455   

Year: 2019 - End-Month of observation: 4
Forecast Year: 2019 -  Forecast Month: 7
Dropping year: 2019  - get series for analysis.
     Month  Year  Anomaly_Qlog       DATE
463      8  2018           NaN 2018-08-01
464      9  2018           NaN 2018-09-01
465     10  2018           NaN 2018-10-01
466     11  2018           NaN 2018-11-01
467     12  2018           NaN 2018-12-01
468      1  2019           NaN 2019-01-01
469      2  2019           NaN 2019-02-01
470      3  2019     -0.449993 2019-03-01
471      4  2019     -1.319303 2019-04-01
There are enough months in the past
Target period: 08-2018 to 04-2019
Target period includes two years

Year: 2019 - End-Month of observation: 5
Forecast Year: 2019 -  Forecast Month: 8
Dropping year: 2019  - get series for analysis.
     Month  Year  Anomaly_Qlog       DATE
464      9  2018           NaN 2018-09-01
465     10  2018           NaN 2018-10-01
466     11  2018           NaN 2018-11-01
467     12  2018           NaN 2018-12-01
468   

Year: 2020 - End-Month of observation: 5
Forecast Year: 2020 -  Forecast Month: 8
Dropping year: 2020  - get series for analysis.
     Month  Year  Anomaly_Qlog       DATE
476      9  2019      0.482405 2019-09-01
477     10  2019      1.729348 2019-10-01
478     11  2019     -0.157611 2019-11-01
479     12  2019     -0.782450 2019-12-01
480      1  2020     -0.385739 2020-01-01
481      2  2020     -0.742382 2020-02-01
482      3  2020     -1.059530 2020-03-01
483      4  2020     -1.188353 2020-04-01
484      5  2020     -1.607545 2020-05-01
There are enough months in the past
Target period: 09-2019 to 05-2020
Target period includes two years

Year: 2020 - End-Month of observation: 6
Forecast Year: 2020 -  Forecast Month: 9
Dropping year: 2020  - get series for analysis.
     Month  Year  Anomaly_Qlog       DATE
477     10  2019      1.729348 2019-10-01
478     11  2019     -0.157611 2019-11-01
479     12  2019     -0.782450 2019-12-01
480      1  2020     -0.385739 2020-01-01
481   

Year: 2021 - End-Month of observation: 6
Forecast Year: 2021 -  Forecast Month: 9
Dropping year: 2021  - get series for analysis.
     Month  Year  Anomaly_Qlog       DATE
489     10  2020     -0.961647 2020-10-01
490     11  2020     -1.797352 2020-11-01
491     12  2020     -1.398219 2020-12-01
492      1  2021     -0.230398 2021-01-01
493      2  2021      0.860497 2021-02-01
494      3  2021      0.710564 2021-03-01
495      4  2021      0.670950 2021-04-01
496      5  2021     -0.087954 2021-05-01
497      6  2021      1.115231 2021-06-01
There are enough months in the past
Target period: 10-2020 to 06-2021
Target period includes two years

Year: 2021 - End-Month of observation: 7
Forecast Year: 2021 -  Forecast Month: 10
Dropping year: 2021  - get series for analysis.
     Month  Year  Anomaly_Qlog       DATE
490     11  2020     -1.797352 2020-11-01
491     12  2020     -1.398219 2020-12-01
492      1  2021     -0.230398 2021-01-01
493      2  2021      0.860497 2021-02-01
494  

Year: 2022 - End-Month of observation: 7
Forecast Year: 2022 -  Forecast Month: 10
Dropping year: 2022  - get series for analysis.
     Month  Year  Anomaly_Qlog       DATE
502     11  2021     -1.153155 2021-11-01
503     12  2021     -1.154836 2021-12-01
504      1  2022      1.141266 2022-01-01
505      2  2022     -0.252152 2022-02-01
506      3  2022     -0.389803 2022-03-01
507      4  2022     -0.613546 2022-04-01
508      5  2022     -0.178636 2022-05-01
509      6  2022     -1.331390 2022-06-01
510      7  2022      0.197415 2022-07-01
There are enough months in the past
Target period: 11-2021 to 07-2022
Target period includes two years

Year: 2022 - End-Month of observation: 8
Forecast Year: 2022 -  Forecast Month: 11
Dropping year: 2022  - get series for analysis.
     Month  Year  Anomaly_Qlog       DATE
503     12  2021     -1.154836 2021-12-01
504      1  2022      1.141266 2022-01-01
505      2  2022     -0.252152 2022-02-01
506      3  2022     -0.389803 2022-03-01
507 

In [41]:
RESULTS = pd.DataFrame()
RESULTS['MONTH_PRED'] = MON_PRED
RESULTS['YEAR_PRED'] = ANO_PRED
RESULTS['MONTH_TARG'] = MON_TARG
RESULTS['YEAR_TARG'] = ANO_TARG
RESULTS['RMSE'] = RMSE
RESULTS['TARG'] = TARG
RESULTS['PRED'] = PRED
RESULTS['OBS_ANOM'] = OBS_ANOM
RESULTS['MEAN_ANOM_HIND'] = MEAN_ANOM_HIND
RESULTS['STD_ANOM_HIND'] = STD_ANOM_HIND
RESULTS['Q1_ANOM_HIND'] = Q1_ANOM_HIND
RESULTS['Q3_ANOM_HIND'] = Q3_ANOM_HIND

In [42]:
RESULTS.iloc[66:80]

,MONTH_PRED,YEAR_PRED,MONTH_TARG,YEAR_TARG,RMSE,TARG,PRED,OBS_ANOM,MEAN_ANOM_HIND,STD_ANOM_HIND,Q1_ANOM_HIND,Q3_ANOM_HIND
66,1,1981,"[2, 3, 4, 5, 6, 7, 8, 9, 10]",2005-2005,0.9126,"[0.31691, 1.12085, 1.20088, 1.37364, 0.79624, ...","[0.49291, -0.2787, 1.68828, 1.21699, 1.5332, 0...",2.42733,0.029435,1.016292,-0.571864,0.582247
67,1,1981,"[2, 3, 4, 5, 6, 7, 8, 9, 10]",2006-2006,1.1219,"[0.31691, 1.12085, 1.20088, 1.37364, 0.79624, ...","[-0.13756, 0.09441, -0.48783, -0.98635, 1.3171...",-0.40592,0.029435,1.016292,-0.571864,0.582247
68,1,1981,"[2, 3, 4, 5, 6, 7, 8, 9, 10]",2007-2007,1.1491,"[0.31691, 1.12085, 1.20088, 1.37364, 0.79624, ...","[-0.67255, 2.33215, 1.95694, 1.71323, -0.21785...",-0.06992,0.029435,1.016292,-0.571864,0.582247
69,1,1981,"[2, 3, 4, 5, 6, 7, 8, 9, 10]",2008-2008,1.4531,"[0.31691, 1.12085, 1.20088, 1.37364, 0.79624, ...","[-0.30374, 0.80334, -0.65346, -0.86467, -0.893...",-0.27512,0.029435,1.016292,-0.571864,0.582247
70,1,1981,"[2, 3, 4, 5, 6, 7, 8, 9, 10]",2009-2009,1.2082,"[0.31691, 1.12085, 1.20088, 1.37364, 0.79624, ...","[0.21308, 0.48277, -0.65186, -1.02821, -0.5822...",1.08240,0.029435,1.016292,-0.571864,0.582247
71,1,1981,"[2, 3, 4, 5, 6, 7, 8, 9, 10]",2010-2010,1.3789,"[0.31691, 1.12085, 1.20088, 1.37364, 0.79624, ...","[2.82837, 0.54933, 0.44626, -0.30553, 0.61267,...",-0.17149,0.029435,1.016292,-0.571864,0.582247
72,1,1981,"[2, 3, 4, 5, 6, 7, 8, 9, 10]",2011-2011,NaN,"[0.31691, 1.12085, 1.20088, 1.37364, 0.79624, ...","[-0.39905, -0.53684, -0.67959, -0.99035, nan, ...",NaN,0.029435,1.016292,-0.571864,0.582247
73,1,1981,"[2, 3, 4, 5, 6, 7, 8, 9, 10]",2012-2012,NaN,"[0.31691, 1.12085, 1.20088, 1.37364, 0.79624, ...","[nan, nan, -0.58655, -1.22073, -1.20696, -0.78...",NaN,0.029435,1.016292,-0.571864,0.582247
74,1,1981,"[2, 3, 4, 5, 6, 7, 8, 9, 10]",2013-2013,NaN,"[0.31691, 1.12085, 1.20088, 1.37364, 0.79624, ...","[-0.27839, nan, nan, nan, -1.17355, nan, nan, ...",NaN,0.029435,1.016292,-0.571864,0.582247
75,1,1981,"[2, 3, 4, 5, 6, 7, 8, 9, 10]",2014-2014,NaN,"[0.31691, 1.12085, 1.20088, 1.37364, 0.79624, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan]",NaN,0.029435,1.016292,-0.571864,0.582247


In [43]:
RESULTS.iloc[1230:1239]

,MONTH_PRED,YEAR_PRED,MONTH_TARG,YEAR_TARG,RMSE,TARG,PRED,OBS_ANOM,MEAN_ANOM_HIND,STD_ANOM_HIND,Q1_ANOM_HIND,Q3_ANOM_HIND
1230,5,1983,"[6, 7, 8, 9, 10, 11, 12, 1, 2]",2010-2011,0.7175,"[1.17573, 0.60522, 0.72602, 0.38415, nan, nan,...","[0.61267, 1.05753, 0.99331, 1.74624, -0.69146,...",-0.99035,0.018355,1.005721,-0.794718,0.727455
1231,5,1983,"[6, 7, 8, 9, 10, 11, 12, 1, 2]",2011-2012,NaN,"[1.17573, 0.60522, 0.72602, 0.38415, nan, nan,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan]",NaN,0.018355,1.005721,-0.794718,0.727455
1232,5,1983,"[6, 7, 8, 9, 10, 11, 12, 1, 2]",2012-2013,NaN,"[1.17573, 0.60522, 0.72602, 0.38415, nan, nan,...","[-1.20696, -0.78242, nan, nan, 1.35065, 1.0568...",NaN,0.018355,1.005721,-0.794718,0.727455
1233,5,1983,"[6, 7, 8, 9, 10, 11, 12, 1, 2]",2013-2014,NaN,"[1.17573, 0.60522, 0.72602, 0.38415, nan, nan,...","[-1.17355, nan, nan, nan, -0.52926, nan, nan, ...",NaN,0.018355,1.005721,-0.794718,0.727455
1234,5,1983,"[6, 7, 8, 9, 10, 11, 12, 1, 2]",2014-2015,NaN,"[1.17573, 0.60522, 0.72602, 0.38415, nan, nan,...","[nan, nan, nan, nan, nan, 1.22796, nan, 0.2048...",NaN,0.018355,1.005721,-0.794718,0.727455
1235,5,1983,"[6, 7, 8, 9, 10, 11, 12, 1, 2]",2015-2016,2.0100,"[1.17573, 0.60522, 0.72602, 0.38415, nan, nan,...","[-1.81426, -2.45869, 0.98308, -1.12454, -0.102...",0.04985,0.018355,1.005721,-0.794718,0.727455
1236,5,1983,"[6, 7, 8, 9, 10, 11, 12, 1, 2]",2016-2017,0.8425,"[1.17573, 0.60522, 0.72602, 0.38415, nan, nan,...","[-0.30178, 1.46443, 0.58962, 0.69366, -0.58094...",0.10773,0.018355,1.005721,-0.794718,0.727455
1237,5,1983,"[6, 7, 8, 9, 10, 11, 12, 1, 2]",2017-2018,1.1998,"[1.17573, 0.60522, 0.72602, 0.38415, nan, nan,...","[-0.85083, 0.16631, 1.7006, 1.62972, -0.10848,...",-0.53768,0.018355,1.005721,-0.794718,0.727455
1238,5,1983,"[6, 7, 8, 9, 10, 11, 12, 1, 2]",2018-2019,NaN,"[1.17573, 0.60522, 0.72602, 0.38415, nan, nan,...","[-0.52017, 0.88362, nan, nan, nan, nan, nan, n...",NaN,0.018355,1.005721,-0.794718,0.727455


In [44]:
RESULTS.tail(10)

,MONTH_PRED,YEAR_PRED,MONTH_TARG,YEAR_TARG,RMSE,TARG,PRED,OBS_ANOM,MEAN_ANOM_HIND,STD_ANOM_HIND,Q1_ANOM_HIND,Q3_ANOM_HIND
20864,12,2022,"[1, 2, 3, 4, 5, 6, 7, 8, 9]",2012-2012,NaN,"[1.14127, -0.25215, -0.3898, -0.61355, -0.1786...","[nan, nan, nan, -0.58655, -1.22073, -1.20696, ...",NaN,0.029552,0.997665,-0.718383,0.508821
20865,12,2022,"[1, 2, 3, 4, 5, 6, 7, 8, 9]",2013-2013,NaN,"[1.14127, -0.25215, -0.3898, -0.61355, -0.1786...","[0.90838, -0.27839, nan, nan, nan, -1.17355, n...",NaN,0.029552,0.997665,-0.718383,0.508821
20866,12,2022,"[1, 2, 3, 4, 5, 6, 7, 8, 9]",2014-2014,NaN,"[1.14127, -0.25215, -0.3898, -0.61355, -0.1786...","[nan, nan, nan, nan, nan, nan, nan, nan, nan]",NaN,0.029552,0.997665,-0.718383,0.508821
20867,12,2022,"[1, 2, 3, 4, 5, 6, 7, 8, 9]",2015-2015,NaN,"[1.14127, -0.25215, -0.3898, -0.61355, -0.1786...","[0.2048, -0.39451, -0.67752, nan, -1.28372, -1...",NaN,0.029552,0.997665,-0.718383,0.508821
20868,12,2022,"[1, 2, 3, 4, 5, 6, 7, 8, 9]",2016-2016,1.5570,"[1.14127, -0.25215, -0.3898, -0.61355, -0.1786...","[0.81419, -0.88705, -1.09663, 2.11815, 0.04985...",-0.31237,0.029552,0.997665,-0.718383,0.508821
20869,12,2022,"[1, 2, 3, 4, 5, 6, 7, 8, 9]",2017-2017,1.6307,"[1.14127, -0.25215, -0.3898, -0.61355, -0.1786...","[0.20097, 0.57783, -0.5459, -0.96938, 0.10773,...",-0.80314,0.029552,0.997665,-0.718383,0.508821
20870,12,2022,"[1, 2, 3, 4, 5, 6, 7, 8, 9]",2018-2018,NaN,"[1.14127, -0.25215, -0.3898, -0.61355, -0.1786...","[0.32454, -0.65973, 0.11227, -0.97901, -0.5376...",NaN,0.029552,0.997665,-0.718383,0.508821
20871,12,2022,"[1, 2, 3, 4, 5, 6, 7, 8, 9]",2019-2019,NaN,"[1.14127, -0.25215, -0.3898, -0.61355, -0.1786...","[nan, nan, -0.44999, -1.3193, 0.05329, 1.71577...",NaN,0.029552,0.997665,-0.718383,0.508821
20872,12,2022,"[1, 2, 3, 4, 5, 6, 7, 8, 9]",2020-2020,1.2570,"[1.14127, -0.25215, -0.3898, -0.61355, -0.1786...","[-0.38574, -0.74238, -1.05953, -1.18835, -1.60...",-1.39822,0.029552,0.997665,-0.718383,0.508821
20873,12,2022,"[1, 2, 3, 4, 5, 6, 7, 8, 9]",2021-2021,1.4085,"[1.14127, -0.25215, -0.3898, -0.61355, -0.1786...","[-0.2304, 0.8605, 0.71056, 0.67095, -0.08795, ...",-1.15484,0.029552,0.997665,-0.718383,0.508821


In [45]:
TTT = pd.DataFrame()
TTT['DATE'] = pd.date_range(datetime.date.today().replace(year=RESULTS.iloc[0]['YEAR_PRED'], month=RESULTS.iloc[0]['MONTH_PRED'], day=1),datetime.date.today().replace(year=RESULTS.iloc[-1]['YEAR_PRED'], month=RESULTS.iloc[-1]['MONTH_PRED'], day=1),freq='MS')
TTT['MONTH_PRED'] = pd.date_range(datetime.date.today().replace(year=RESULTS.iloc[0]['YEAR_PRED'], month=RESULTS['MONTH_PRED'][0], day=1),datetime.date.today().replace(year=RESULTS.iloc[-1]['YEAR_PRED'], month=RESULTS.iloc[-1]['MONTH_PRED'], day=1),freq='MS').month
TTT['YEAR_PRED'] = pd.date_range(datetime.date.today().replace(year=RESULTS.iloc[0]['YEAR_PRED'], month=RESULTS['MONTH_PRED'][0], day=1),datetime.date.today().replace(year=RESULTS.iloc[-1]['YEAR_PRED'], month=RESULTS.iloc[-1]['MONTH_PRED'], day=1),freq='MS').year
TTT['MONTH_PRED_cal'] = TTT['MONTH_PRED'].apply(lambda x: calendar.month_abbr[x])

In [46]:
TTT.tail(5)

,DATE,MONTH_PRED,YEAR_PRED,MONTH_PRED_cal
500,2022-08-01,8,2022,Aug
501,2022-09-01,9,2022,Sep
502,2022-10-01,10,2022,Oct
503,2022-11-01,11,2022,Nov
504,2022-12-01,12,2022,Dec


In [47]:
best_n = 5
INDEX_best_n = []
RMSE_best_n = []
w_best_n = []
OBS_ANOM_best_n = []

MEAN_ANOM_HIND_best_n = []
STD_ANOM_HIND_best_n = []
Q1_ANOM_HIND_best_n = []
Q3_ANOM_HIND_best_n = []

for y in np.sort(TTT['YEAR_PRED'].unique()):
    #print(y)
    for m in np.sort(TTT['MONTH_PRED'].unique()):
        
        ## Get index observerved anomaly value series best n RMSE sorted
        AUX_1 = RESULTS.query('MONTH_PRED == @m & YEAR_PRED == @y').sort_values(by=['RMSE']).head(best_n).index
        AUX_2 = AUX_1.values.tolist()
        if AUX_2:
            INDEX_best_n.append(AUX_2)
        else:
            pass
        
        ## Get predicted anomaly value series best n RMSE sorted
        AUX_3 = RESULTS.query('MONTH_PRED == @m & YEAR_PRED == @y').sort_values(by=['RMSE']).head(best_n)
        AUX_4 = AUX_3['RMSE'].values.tolist()
#         AUX_5 = AUX_3['OBS_ANOM'].values.tolist()
        if AUX_4:
            RMSE_best_n.append(AUX_4)
            #Calculation of weights
            w_best_n.append(((1/np.array(AUX_4)) / (1/np.array(AUX_4)).sum()).round(6))
        else:
            pass

        ## Get observerved anomaly value series best n RMSE sorted
        AUX_5 = AUX_3['OBS_ANOM'].values.tolist()
        if AUX_5:
            OBS_ANOM_best_n.append(AUX_5)    
        else:
            pass
        
        ## Get mean anomaly of hindcast series best n RMSE sorted
        AUX_6 = AUX_3['MEAN_ANOM_HIND'].values.tolist()              
        if AUX_6:
            MEAN_ANOM_HIND_best_n.append(AUX_6[0])    
        else:
            pass
        
        ## Get standard deviation anomaly of hindcast series best n RMSE sorted
        AUX_7 = AUX_3['STD_ANOM_HIND'].values.tolist()       
        if AUX_7:
            STD_ANOM_HIND_best_n.append(AUX_7[0])    
        else:
            pass        
        
        ## Get Q1 anomaly of hindcast series best n RMSE sorted
        AUX_8 = AUX_3['Q1_ANOM_HIND'].values.tolist()       
        if AUX_8:
            Q1_ANOM_HIND_best_n.append(AUX_8[0])    
        else:
            pass        

        ## Get Q3 anomaly of hindcast series best n RMSE sorted
        AUX_9 = AUX_3['Q3_ANOM_HIND'].values.tolist()       
        if AUX_8:
            Q3_ANOM_HIND_best_n.append(AUX_9[0])    
        else:
            pass  

In [48]:
TTT['INDEX_sort_best_n'] = INDEX_best_n
TTT['RMSE_sort_best_n'] = RMSE_best_n
TTT['Weights_sort_best_n'] = w_best_n
TTT['OBS_ANOM_sort_best_n'] = OBS_ANOM_best_n
TTT['MEAN_ANOM_HIND_best_n'] = MEAN_ANOM_HIND_best_n
TTT['STD_ANOM_HIND_best_n'] = STD_ANOM_HIND_best_n
TTT['Q1_ANOM_HIND_best_n'] = Q1_ANOM_HIND_best_n
TTT['Q3_ANOM_HIND_best_n'] = Q3_ANOM_HIND_best_n

In [49]:
TTT.head(15)

,DATE,MONTH_PRED,YEAR_PRED,MONTH_PRED_cal,INDEX_sort_best_n,RMSE_sort_best_n,Weights_sort_best_n,OBS_ANOM_sort_best_n,MEAN_ANOM_HIND_best_n,STD_ANOM_HIND_best_n,Q1_ANOM_HIND_best_n,Q3_ANOM_HIND_best_n
0,1980-12-01,12,1980,Dec,"[4, 40, 21, 0, 14]","[0.7381, 0.9338, 0.9902, 1.0408, 1.081]","[0.254596, 0.201239, 0.189777, 0.180551, 0.173...","[-0.7507, -1.15484, 1.83201, 0.39842, -1.02869]",-0.026612,1.000810,-0.753450,0.419850
1,1981-01-01,1,1981,Jan,"[46, 63, 82, 66, 54]","[0.6793, 0.7116, 0.8805, 0.9126, 0.9566]","[0.239201, 0.228344, 0.184542, 0.178051, 0.169...","[-0.67026, 1.66245, 1.14127, 2.42733, 1.29785]",0.029435,1.016292,-0.571864,0.582247
2,1981-02-01,2,1981,Feb,"[88, 105, 87, 124, 108]","[0.4964, 0.704, 0.8979, 0.9295, 0.9444]","[0.30142, 0.212535, 0.166638, 0.160973, 0.158434]","[-0.37071, 0.95838, -1.25926, -0.25215, -0.13756]",0.011885,1.011422,-0.649781,0.233294
3,1981-03-01,3,1981,Mar,"[147, 130, 150, 139, 126]","[0.6006, 0.745, 0.9166, 0.9168, 0.9204]","[0.265317, 0.213892, 0.173849, 0.173811, 0.173...","[0.21482, -0.89846, 0.09441, -0.02834, -0.37581]",0.049073,0.991664,-0.619743,0.506735
4,1981-04-01,4,1981,Apr,"[178, 172, 179, 181, 189]","[0.6748, 0.7262, 0.8123, 0.8252, 0.8406]","[0.228287, 0.212129, 0.189644, 0.18668, 0.18326]","[-0.10971, -0.56918, 0.94189, 1.1429, -0.68975]",0.048082,0.987980,-0.638065,0.607177
5,1981-05-01,5,1981,May,"[219, 234, 229, 213, 232]","[0.5838, 0.7343, 0.7349, 0.7584, 0.8103]","[0.245117, 0.194878, 0.194719, 0.188686, 0.1766]","[0.89762, 1.71323, 1.45493, 0.31567, 1.21699]",-0.045121,0.969551,-0.794718,0.675120
6,1981-06-01,6,1981,Jun,"[260, 263, 273, 254, 285]","[0.5721, 0.6948, 0.7525, 0.7783, 0.7876]","[0.247212, 0.203555, 0.187947, 0.181716, 0.179...","[0.51762, 0.30984, 1.5332, 0.50837, -0.85083]",0.004089,1.012392,-0.585955,0.618781
7,1981-07-01,7,1981,Jul,"[301, 304, 326, 295, 291]","[0.5722, 0.7197, 0.7889, 0.7898, 0.8295]","[0.254151, 0.202064, 0.184339, 0.184129, 0.175...","[0.28514, 0.49932, 0.16631, 0.09568, 0.60522]",-0.008401,1.011642,-0.399261,0.610187
8,1981-08-01,8,1981,Aug,"[342, 355, 332, 343, 345]","[0.6404, 0.8326, 0.8717, 0.8958, 0.898]","[0.254334, 0.195622, 0.186848, 0.181821, 0.181...","[-0.70814, -1.10505, 0.72602, 0.02873, -1.40723]",-0.025762,1.001655,-0.576282,0.592820
9,1981-09-01,9,1981,Sep,"[383, 386, 384, 395, 377]","[0.5551, 0.7973, 0.8422, 0.8646, 0.9143]","[0.277431, 0.193155, 0.182857, 0.17812, 0.168437]","[0.5321, -1.77771, 0.6566, 0.27023, 0.92787]",-0.014555,1.010203,-0.642866,0.556999


In [50]:
TTT.tail(15)

,DATE,MONTH_PRED,YEAR_PRED,MONTH_PRED_cal,INDEX_sort_best_n,RMSE_sort_best_n,Weights_sort_best_n,OBS_ANOM_sort_best_n,MEAN_ANOM_HIND_best_n,STD_ANOM_HIND_best_n,Q1_ANOM_HIND_best_n,Q3_ANOM_HIND_best_n
490,2021-10-01,10,2021,Oct,"[20263, 20266, 20282, 20278, 20281]","[0.8756, 0.9315, 0.9466, 1.0321, 1.0492]","[0.219882, 0.206687, 0.20339, 0.186541, 0.183501]","[0.0175, 1.33573, 1.3713, -0.66635, -1.16394]",0.027208,0.999433,-0.576807,0.782336
491,2021-11-01,11,2021,Nov,"[20307, 20319, 20299, 20323, 20318]","[0.7084, 0.8784, 0.928, 0.9443, 1.0087]","[0.248614, 0.200499, 0.189782, 0.186506, 0.174...","[1.56234, -0.85743, 0.63389, 1.29656, 0.16546]",0.031166,0.994908,-0.674494,0.682502
492,2021-12-01,12,2021,Dec,"[20349, 20361, 20348, 20360, 20341]","[0.6937, 0.8061, 0.8386, 0.845, 0.9216]","[0.234663, 0.201942, 0.194116, 0.192646, 0.176...","[2.35254, -0.26371, -0.66184, -0.25158, -0.7507]",0.032079,0.994690,-0.718383,0.508821
493,2022-01-01,1,2022,Jan,"[20403, 20404, 20401, 20378, 20397]","[0.8004, 0.8278, 0.8468, 0.8805, 0.8994]","[0.212266, 0.20524, 0.200635, 0.192956, 0.188902]","[2.42733, -0.40592, 0.22036, -0.30818, -0.75205]",-0.008709,1.000513,-0.571864,0.287036
494,2022-02-01,2,2022,Feb,"[20439, 20445, 20450, 20432, 20420]","[0.7586, 0.7971, 0.8089, 0.9191, 0.9295]","[0.220701, 0.210041, 0.206977, 0.18216, 0.180122]","[-1.08268, -0.13756, -0.39905, 1.70298, 0.5119]",0.032535,1.013679,-0.649781,0.310361
495,2022-03-01,3,2022,Mar,"[20481, 20487, 20474, 20492, 20502]","[0.707, 0.7841, 0.8213, 0.8275, 0.8959]","[0.226962, 0.204645, 0.195375, 0.193912, 0.179...","[-1.34189, 0.09441, -0.43461, -0.53684, 0.71056]",0.049761,0.991360,-0.619743,0.506735
496,2022-04-01,4,2022,Apr,"[20544, 20529, 20540, 20516, 20534]","[0.7922, 0.8268, 0.9133, 0.9317, 0.9331]","[0.221003, 0.211754, 0.191699, 0.187913, 0.187...","[0.67095, -0.48783, -0.96938, 0.94189, -0.67959]",0.059942,0.982513,-0.628347,0.607177
497,2022-05-01,5,2022,May,"[20570, 20585, 20575, 20581, 20564]","[0.7042, 0.7131, 0.9316, 0.9532, 0.9706]","[0.237658, 0.234692, 0.179647, 0.175576, 0.172...","[-0.98635, -0.08795, -0.99035, 0.10773, 2.06476]",0.004466,1.012325,-0.794718,0.727455
498,2022-06-01,6,2022,Jun,"[20611, 20614, 20626, 20622, 20623]","[0.7089, 0.78, 0.7822, 0.8295, 0.9512]","[0.2265, 0.205853, 0.205274, 0.193569, 0.168803]","[1.31711, -0.58229, 1.11523, -0.85083, -0.52017]",0.033285,0.989471,-0.525142,0.618781
499,2022-07-01,7,2022,Jul,"[20663, 20652, 20655, 20667, 20664]","[0.648, 0.6947, 0.776, 0.8816, 0.9539]","[0.23911, 0.223036, 0.199669, 0.175753, 0.162432]","[0.16631, 0.21453, 1.06729, -0.1296, 0.88362]",-0.005062,1.012553,-0.399261,0.610187


In [51]:
Pred_Anom = []
for i in TTT.index:
    Pred_Anom.append((TTT['Weights_sort_best_n'][i] * TTT['OBS_ANOM_sort_best_n'][i]).sum())
    
TTT['Frcst_Anom'] = Pred_Anom

In [52]:
TTT

,DATE,MONTH_PRED,YEAR_PRED,MONTH_PRED_cal,INDEX_sort_best_n,RMSE_sort_best_n,Weights_sort_best_n,OBS_ANOM_sort_best_n,MEAN_ANOM_HIND_best_n,STD_ANOM_HIND_best_n,Q1_ANOM_HIND_best_n,Q3_ANOM_HIND_best_n,Frcst_Anom
0,1980-12-01,12,1980,Dec,"[4, 40, 21, 0, 14]","[0.7381, 0.9338, 0.9902, 1.0408, 1.081]","[0.254596, 0.201239, 0.189777, 0.180551, 0.173...","[-0.7507, -1.15484, 1.83201, 0.39842, -1.02869]",-0.026612,1.000810,-0.753450,0.419850,-0.182740
1,1981-01-01,1,1981,Jan,"[46, 63, 82, 66, 54]","[0.6793, 0.7116, 0.8805, 0.9126, 0.9566]","[0.239201, 0.228344, 0.184542, 0.178051, 0.169...","[-0.67026, 1.66245, 1.14127, 2.42733, 1.29785]",0.029435,1.016292,-0.571864,0.582247,1.082539
2,1981-02-01,2,1981,Feb,"[88, 105, 87, 124, 108]","[0.4964, 0.704, 0.8979, 0.9295, 0.9444]","[0.30142, 0.212535, 0.166638, 0.160973, 0.158434]","[-0.37071, 0.95838, -1.25926, -0.25215, -0.13756]",0.011885,1.011422,-0.649781,0.233294,-0.180274
3,1981-03-01,3,1981,Mar,"[147, 130, 150, 139, 126]","[0.6006, 0.745, 0.9166, 0.9168, 0.9204]","[0.265317, 0.213892, 0.173849, 0.173811, 0.173...","[0.21482, -0.89846, 0.09441, -0.02834, -0.37581]",0.049073,0.991664,-0.619743,0.506735,-0.188755
4,1981-04-01,4,1981,Apr,"[178, 172, 179, 181, 189]","[0.6748, 0.7262, 0.8123, 0.8252, 0.8406]","[0.228287, 0.212129, 0.189644, 0.18668, 0.18326]","[-0.10971, -0.56918, 0.94189, 1.1429, -0.68975]",0.048082,0.987980,-0.638065,0.607177,0.119792
...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,2022-08-01,8,2022,Aug,"[20704, 20693, 20696, 20708, 20684]","[0.6242, 0.6486, 0.7515, 0.8477, 0.8907]","[0.236399, 0.227506, 0.196355, 0.174072, 0.165...","[1.7006, 0.33147, -0.36598, -0.00509, -0.16317]",0.035617,0.990098,-0.401450,0.626634,0.377651
501,2022-09-01,9,2022,Sep,"[20745, 20737, 20725, 20736, 20716]","[0.6088, 0.7106, 0.9544, 1.0262, 1.0834]","[0.273986, 0.234735, 0.174772, 0.162544, 0.153...","[1.62972, -0.33655, -1.23853, -0.98884, -1.1859]",0.054879,0.956010,-0.504781,0.556999,-0.192254
502,2022-10-01,10,2022,Oct,"[20786, 20777, 20778, 20757, 20775]","[0.5922, 0.7105, 0.7657, 0.9879, 1.0784]","[0.266276, 0.22194, 0.20594, 0.15962, 0.146224]","[-0.10848, -1.16394, 1.3713, -1.28033, -0.12469]",0.047641,0.969089,-0.576807,0.782336,-0.227404
503,2022-11-01,11,2022,Nov,"[20818, 20814, 20819, 20797, 20798]","[0.6981, 0.7734, 0.8207, 0.9875, 1.0369]","[0.241929, 0.218374, 0.205788, 0.171028, 0.16288]","[-1.38478, 0.16546, 1.29656, -0.41519, -0.54493]",0.029675,0.996687,-0.674494,0.682502,-0.191837


In [53]:
TTT['Frcst_Anom_Rest'] = ( TTT['Frcst_Anom'] - TTT['MEAN_ANOM_HIND_best_n'] ) / TTT['STD_ANOM_HIND_best_n']

In [54]:
HIND_FORECAST_ANALOGUE = pd.DataFrame()
HIND_FORECAST_ANALOGUE['DATE'] = TTT['DATE']
HIND_FORECAST_ANALOGUE['MONTH_PRED'] = TTT['MONTH_PRED']
HIND_FORECAST_ANALOGUE['YEAR_PRED'] = TTT['YEAR_PRED']
HIND_FORECAST_ANALOGUE['FORECAST'] = TTT['Frcst_Anom_Rest']

In [55]:
ANOMALY_ANALOGUE_TS

,Month,Year,Anomaly_Qlog,DATE
0,1,1980,-2.123750,1980-01-01
1,2,1980,0.316909,1980-02-01
2,3,1980,1.120851,1980-03-01
3,4,1980,1.200881,1980-04-01
4,5,1980,1.373637,1980-05-01
...,...,...,...,...
511,8,2022,-1.282212,2022-08-01
512,9,2022,-1.975643,2022-09-01
513,10,2022,-1.762722,2022-10-01
514,11,2022,-1.097971,2022-11-01


In [56]:
HIND_FORECAST_ANALOGUE

,DATE,MONTH_PRED,YEAR_PRED,FORECAST
0,1980-12-01,12,1980,-0.156002
1,1981-01-01,1,1981,1.036221
2,1981-02-01,2,1981,-0.189989
3,1981-03-01,3,1981,-0.239827
4,1981-04-01,4,1981,0.072582
...,...,...,...,...
500,2022-08-01,8,2022,0.345455
501,2022-09-01,9,2022,-0.258504
502,2022-10-01,10,2022,-0.283818
503,2022-11-01,11,2022,-0.222248


In [57]:
OBSERVED_AND_HIND_FORECAST_ANALOGUE = pd.merge(ANOMALY_ANALOGUE_TS, HIND_FORECAST_ANALOGUE[['DATE','FORECAST']], how='left', on='DATE')

In [58]:
OBSERVED_AND_HIND_FORECAST_ANALOGUE.head(31)

,Month,Year,Anomaly_Qlog,DATE,FORECAST
0,1,1980,-2.123750,1980-01-01,NaN
1,2,1980,0.316909,1980-02-01,NaN
2,3,1980,1.120851,1980-03-01,NaN
3,4,1980,1.200881,1980-04-01,NaN
4,5,1980,1.373637,1980-05-01,NaN
5,6,1980,0.796236,1980-06-01,NaN
6,7,1980,0.669569,1980-07-01,NaN
7,8,1980,0.585540,1980-08-01,NaN
8,9,1980,-0.481964,1980-09-01,NaN
9,10,1980,0.640823,1980-10-01,NaN


In [59]:
OBSERVED_AND_HIND_FORECAST_ANALOGUE.iloc[45:55]

,Month,Year,Anomaly_Qlog,DATE,FORECAST
45,10,1983,1.573028,1983-10-01,0.525646
46,11,1983,1.199867,1983-11-01,0.254514
47,12,1983,-0.256680,1983-12-01,-0.152365
48,1,1984,0.727208,1984-01-01,-0.045369
49,2,1984,2.454052,1984-02-01,-0.615199
50,3,1984,1.193772,1984-03-01,0.274948
51,4,1984,-0.160803,1984-04-01,0.167929
52,5,1984,0.722645,1984-05-01,0.846196
53,6,1984,1.159690,1984-06-01,0.723420
54,7,1984,1.575762,1984-07-01,0.200414


In [60]:
OBSERVED_AND_HIND_FORECAST_ANALOGUE.tail(25)

,Month,Year,Anomaly_Qlog,DATE,FORECAST
491,12,2020,-1.398219,2020-12-01,0.580222
492,1,2021,-0.230398,2021-01-01,0.657106
493,2,2021,0.860497,2021-02-01,0.554389
494,3,2021,0.710564,2021-03-01,-0.129103
495,4,2021,0.670950,2021-04-01,-0.491532
496,5,2021,-0.087954,2021-05-01,0.421934
497,6,2021,1.115231,2021-06-01,-0.540535
498,7,2021,-0.129605,2021-07-01,-0.192798
499,8,2021,-0.005090,2021-08-01,-0.407103
500,9,2021,0.059747,2021-09-01,0.034922


### Save  OBSERVED_AND_HIND_FORECAST_ANALOGUE

In [61]:
filename_1 = 'santalucia_caudales_HINDCAST_ANALOGUE_abr2023' + '_FL_' + str(FORECAST_LENGTH) + '_DANA_' + str(DANA)

In [62]:
filename_1

'santalucia_caudales_HINDCAST_abr2023_FL_3_DANA_9'

In [63]:
OBSERVED_AND_HIND_FORECAST_ANALOGUE.to_csv('../data/' + filename_1 + '.csv', index = False)

In [64]:
filename_2 = 'santalucia_caudales_HINDCAST_ANALOGUE_des_abr2023' + '_FL_' + str(FORECAST_LENGTH) + '_DANA_' + str(DANA)

In [65]:
TTT.to_csv('../data/' + filename_2 + '.csv', index = False)

In [66]:
filename_3 = 'santalucia_caudales_HINDCAST_ANALOGUE_res_abr2023' + '_FL_' + str(FORECAST_LENGTH) + '_DANA_' + str(DANA)
RESULTS.to_csv('../data/' + filename_3 + '.csv', index = False)